<a href="https://colab.research.google.com/github/Vaibhav-sa30/Workshop-Notes/blob/main/Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install squarify
!pip install contractions
!pip install opendatasets

In [47]:
import opendatasets as od
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import contractions
import warnings
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
warnings.filterwarnings("ignore")


In [106]:
dataset_url = "https://www.kaggle.com/datasets/rohitsahoo/sales-forecasting"
od.download(dataset_url)

Skipping, found downloaded files in "./sales-forecasting" (use force=True to force download)


In [107]:
data_dir = '/content/sales-forecasting'

In [108]:
!ls -lh {data_dir}

total 2.1M
-rw-r--r-- 1 root root 2.1M Oct 16 12:45 train.csv


In [109]:
train = pd.read_csv('/content/sales-forecasting/train.csv')

In [110]:
train.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [111]:
train.describe()

,Row ID,Postal Code,Sales
count,9800.000000,9789.000000,9800.000000
mean,4900.500000,55273.322403,230.769059
std,2829.160653,32041.223413,626.651875
min,1.000000,1040.000000,0.444000
25%,2450.750000,23223.000000,17.248000
50%,4900.500000,58103.000000,54.490000
75%,7350.250000,90008.000000,210.605000
max,9800.000000,99301.000000,22638.480000


In [112]:
train['Order Date'] = pd.to_datetime(train['Order Date'], dayfirst = True)

In [113]:
train['Ship Date'] = pd.to_datetime(train['Ship Date'], dayfirst = True)

In [114]:
train['Order year'] = train['Order Date'].dt.year

In [115]:
train['Ship year'] = train['Ship Date'].dt.year

In [116]:
train['Ship year'].equals(train['Order year'])

False

In [117]:
train['Ship Mode'].unique()

array(['Second Class', 'Standard Class', 'First Class', 'Same Day'],
      dtype=object)

In [118]:
train['Ship Mode'].nunique()

4

In [119]:
train['Ship Mode'].value_counts()

Standard Class    5859
Second Class      1902
First Class       1501
Same Day           538
Name: Ship Mode, dtype: int64

In [120]:
train['Customer ID'].value_counts()

WB-21850    35
MA-17560    34
PP-18955    34
JL-15835    33
CK-12205    32
            ..
JR-15700     1
CJ-11875     1
SC-20845     1
RE-19405     1
AO-10810     1
Name: Customer ID, Length: 793, dtype: int64

In [121]:
train['Customer ID'].nunique()

793

793 customers are there

In [122]:
train['Customer Name'].nunique()

793

In [123]:
print('Number of unique postal codes:',train['Postal Code'].nunique())
print(train['Postal Code'].value_counts().index)

Number of unique postal codes: 626
Float64Index([10035.0, 10024.0, 10009.0, 94122.0, 10011.0, 94110.0, 98105.0,
              19134.0, 90049.0, 98103.0,
              ...
              90660.0, 33458.0, 72756.0, 68801.0, 98002.0, 32935.0, 76248.0,
              32127.0, 32503.0, 72762.0],
             dtype='float64', length=626)


I want to do binning

In [124]:
print(train['Postal Code'].value_counts().index[:8]


SyntaxError: ignored

In [125]:

train['Postal Code'][~train['Postal Code'].isin(train['Postal Code'].value_counts().index[:8])] = 'other'

In [126]:
train['Postal Code'].value_counts()

other      8223
10035.0     253
10024.0     225
10009.0     220
94122.0     195
10011.0     193
94110.0     166
98105.0     165
19134.0     160
Name: Postal Code, dtype: int64

In [127]:
train['Postal Code'] = train['Postal Code'].astype(str)

Do the same for other columns

In [128]:
train['City'][~train['City'].isin(train['City'].value_counts().index[:8])] = 'ohters'
train['State'][~train['State'].isin(train['State'].value_counts().index[:8])] = 'ohters'
train['Sub-Category'][~train['Sub-Category'].isin(train['Sub-Category'].value_counts().index[:8])] = 'others'

In [129]:
train.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Order year', 'Ship year'],
      dtype='object')

In [130]:
columns_to_drop = ['Row ID','Order ID', 'Order Date', 'Ship Date','Customer ID',
       'Customer Name','Product Name']

In [131]:
train = train.drop(columns_to_drop, axis = 1)
train

,Ship Mode,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Sales,Order year,Ship year
0,Second Class,Consumer,United States,ohters,ohters,other,South,FUR-BO-10001798,Furniture,others,261.9600,2017,2017
1,Second Class,Consumer,United States,ohters,ohters,other,South,FUR-CH-10000454,Furniture,Chairs,731.9400,2017,2017
2,Second Class,Corporate,United States,Los Angeles,California,other,West,OFF-LA-10000240,Office Supplies,others,14.6200,2017,2017
3,Standard Class,Consumer,United States,ohters,Florida,other,South,FUR-TA-10000577,Furniture,others,957.5775,2016,2016
4,Standard Class,Consumer,United States,ohters,Florida,other,South,OFF-ST-10000760,Office Supplies,Storage,22.3680,2016,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,Standard Class,Corporate,United States,Chicago,Illinois,other,Central,OFF-BI-10003429,Office Supplies,Binders,3.7980,2017,2017
9796,Standard Class,Corporate,United States,ohters,Ohio,other,East,OFF-AR-10001374,Office Supplies,Art,10.3680,2016,2016
9797,Standard Class,Corporate,United States,ohters,Ohio,other,East,TEC-PH-10004977,Technology,Phones,235.1880,2016,2016
9798,Standard Class,Corporate,United States,ohters,Ohio,other,East,TEC-PH-10000912,Technology,Phones,26.3760,2016,2016


In [132]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Ship Mode     9800 non-null   object 
 1   Segment       9800 non-null   object 
 2   Country       9800 non-null   object 
 3   City          9800 non-null   object 
 4   State         9800 non-null   object 
 5   Postal Code   9800 non-null   object 
 6   Region        9800 non-null   object 
 7   Product ID    9800 non-null   object 
 8   Category      9800 non-null   object 
 9   Sub-Category  9800 non-null   object 
 10  Sales         9800 non-null   float64
 11  Order year    9800 non-null   int64  
 12  Ship year     9800 non-null   int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 995.4+ KB


In [133]:
X_train, X_test = train_test_split(train, test_size = 0.2, random_state = 42)

In [135]:
X_train.shape, X_test.shape

((7840, 13), (1960, 13))

In [139]:
X = X_train.drop(['Sales'], axis = 1)
y_train  = X_train['Sales']

In [140]:
X_t = X_test.drop(['Sales'], axis = 1)
y_test  = X_test['Sales']

In [141]:
X.shape, y_train.shape, X_t.shape, y_test.shape

((7840, 12), (7840,), (1960, 12), (1960,))

In [144]:
Cat_features = X.select_dtypes(include = 'object').columns
Num_features = X.select_dtypes(include = ['int', 'float']).columns

In [146]:
CT = ColumnTransformer([('Encode', OneHotEncoder(), Cat_features),
                        ('Scaling', MinMaxScaler(), Num_features), remainder = 'passthrough'])

SyntaxError: ignored